In [ ]:
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import tensorflow_text
import pandas as pd
import numpy as np
import nltk
import re
import os
%matplotlib inline

In [3]:
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.corpus import wordnet
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abulf\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abulf\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\abulf\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\abulf\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [4]:
import sys
from pathlib import Path
import os
sys.path.append(str(Path(os.getcwd()).parents[1]))
from Helper_Functions import common_utils
directory_to_extract_to = os.getcwd()

common_utils.load_data_from_one_drive(directory_to_extract_to, "nlp_paths", "Amazon_sentimental_analysis")

Extracting : 100%|██████████| 92/92 [00:03<00:00, 28.92it/s] 


In [ ]:
def clean_sentence(sentence: str) -> list:
  # Remove the review tag of HTML
  tags = re.compile("(<review_text>|<\/review_text>)")
  sentence = re.sub(tags, '', sentence)
  sentence = sentence.lower()
  # Remove emails and urls
  email_urls = re.compile("(\bhttp.+? | \b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b)")
  sentence = re.sub(email_urls, '', sentence)
  # Some used '@' to hide offensive words (bla -> bl@)
  ats = re.compile('@')
  sentence = re.sub(ats, 'a', sentence)
  # Remove Punctuation 
  #punc = re.compile("[^\w\s(\w+\-\w+)]")
  #punc = re.compile("[!\"\#\$\%\&\'\(\)\*\+,\-\.\/\:;<=>\?\[\\\]\^_`\{\|\}\~]")
  sentence = re.sub(r'[^\w\s]', '', sentence)
  # Remove digits
  pattern = r'[0-9]'
  sentence = re.sub(pattern, '', sentence)
  # Remove stopwords and tokenize
  # sentence = sentence.split(sep=' ')
  sentence = word_tokenize(sentence)
  sentence = [word for word in sentence if not word in stopwords.words()]
  #lemmatizer = WordNetLemmatizer()
  #sentence = [lemmatizer.lemmatize(word) for word in sentence]
  return sentence


In [ ]:
path = "/sorted_data_acl/"#Here I used the Relative path if this cell shows error then try to use the absolute path I gurantee it does not show any error  
regex_review = re.compile("<review_text>.+?<\/review_text>", flags=re.DOTALL)

Collections_test = defaultdict(list)
# Training Data
folders = ["books","dvd","electronics","kitchen_&_housewares","music"]
#x_train = list()
#y_train = list()
negative = 0
positive = 1
print('Reading Train Data')
for folder in folders:
  temp = open(path+folder+"/negative.review", 'r').read() # Read the file
  temp = re.findall(regex_review, temp) # Get reviews
  print("Reading",len(temp),"Negative reviews from",folder)
  #print(temp)
  for sentence in temp:
    list = []
    sentences = clean_sentence(sentence)
    sentences = ' '.join(map(str, sentences))
    list.append(sentences)
    list.append(negative)
    Collections_test[0].append(list)
    #print(Collections)


  temp = open(path+folder+"/positive.review", 'r').read() # Read the file
  temp = re.findall(regex_review, temp) # Get reviews
  print("Reading",len(temp),"Positive reviews from",folder)
  for sentence in temp:
    list = []
    sentences = clean_sentence(sentence)
    sentences = ' '.join(map(str, sentences))
    list.append(sentences)
    list.append(positive)
    Collections_test[0].append(list)
    #print(Collections) 

In [ ]:
#ddf = []
#for key in Collections.keys():
df = pd.DataFrame(Collections_test[0], columns = ['Review', 'Sentiment'])
print(df.head(5))
print(df.tail(5))
    #ddf.append(df)

In [ ]:
df.to_csv('processed_data.csv', index=False)

In [ ]:
'''train_df = df.sample(frac=1, random_state=1)
train_df.reset_index(drop=True, inplace=True)

split_index_1 = int(len(train_df) * 0.7)
split_index_2 = int(len(train_df) * 0.85)

train_dff, val_df, test_df = train_df[:7500], train_df[7500:9000], train_df[9000:]

len(train_dff), len(val_df), len(test_df)'''